In [ ]:
!pip install --upgrade sanpy
!pip install yfinance
!pip install cryptocmd
!pip install sanpy

In [43]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [44]:
#Get historical Bitcoin data
from cryptocmd import CmcScraper
bitcoin=CmcScraper('BTC')
btc_df=bitcoin.get_dataframe().sort_values('Date').reset_index(drop=True)
btc_df['Date']=pd.to_datetime(btc_df['Date'],infer_datetime_format=True)
btc_df.set_index('Date',inplace=True)
btc_df=btc_df[['Close','Volume']]
btc_df.columns=['C_btc','V_btc']

In [45]:
btc_df.head()

,C_btc,V_btc
Date,,
2013-04-28,134.210007,0.0
2013-04-29,144.539993,0.0
2013-04-30,139.000000,0.0
2013-05-01,116.989998,0.0
2013-05-02,105.209999,0.0


In [46]:
btc_df.tail()

,C_btc,V_btc
Date,,
2022-04-22,39740.319854,2.801172e+10
2022-04-23,39486.728711,1.613802e+10
2022-04-24,39469.293556,1.796440e+10
2022-04-25,40458.308965,3.544573e+10
2022-04-26,38117.462508,3.456909e+10


In [48]:
#Get historical SP500,NASDAQ,Dow Jones, Gold Price
import yfinance as yf
sp500_df=yf.Ticker('^GSPC').history(start='2013-04-28')[['Close','Volume']]
sp500_df.columns=['C_sp500','V_sp500']
nasdaq_df=yf.Ticker('NDAQ').history(start='2013-04-28')[['Close','Volume']]
nasdaq_df.columns=['C_nasdaq_df','V_nasdaq_df']
dow_df=yf.Ticker('^DJI').history(start='2013-04-28')[['Close','Volume']]
dow_df.columns=['C_dow_df','V_dow_df']
gold_df=yf.Ticker('GC=F').history(start='2013-04-28')[['Close','Volume']]
gold_df.columns=['C_gold_df','V_gold_df']

In [49]:
#Get important bitcoin metrics
import san
metrics=["active_addresses_24h/bitcoin",'percent_of_total_supply_on_exchanges/bitcoin','social_dominance_total/bitcoin',
'sentiment_positive_total/bitcoin','sentiment_negative_total/bitcoin','sentiment_volume_consumed_total/bitcoin','active_deposits/bitcoin','active_withdrawals/bitcoin',
'circulation/bitcoin','daily_active_addresses/bitcoin','mean_age/bitcoin','mvrv_long_short_diff_usd/bitcoin','mvrv_usd/bitcoin','network_growth/bitcoin','realized_value_usd/bitcoin',
'velocity/bitcoin']

metrics_df=[]
cols=[]
for m in metrics:
    try:
        m_df = san.get(m,
        from_date='2013-04-28',
        interval="1d"
        )
        cols.append(m)
        metrics_df.append(m_df)
    except:
        pass
    
null_index=[]
for i,x in enumerate(metrics_df):
    if x.empty:
        null_index.append(i)

cont=0        
for z in null_index:
    
    if cont==0:
        del cols[z]    
        cont+=1
    else:
        del cols[z-cont]
        cont+=1
        
metrics_df=pd.concat(metrics_df,axis=1)
metrics_df.columns=cols
    

In [50]:
metrics_df.isnull().sum()

active_addresses_24h/bitcoin                       0
percent_of_total_supply_on_exchanges/bitcoin    2586
social_dominance_total/bitcoin                  2586
sentiment_positive_total/bitcoin                2586
sentiment_negative_total/bitcoin                2586
sentiment_volume_consumed_total/bitcoin         2586
circulation/bitcoin                                1
daily_active_addresses/bitcoin                     1
mean_age/bitcoin                                2586
mvrv_long_short_diff_usd/bitcoin                2586
mvrv_usd/bitcoin                                2951
realized_value_usd/bitcoin                      2951
velocity/bitcoin                                2586
dtype: int64

In [52]:
#Drop columns with 2951 nan
metrics_df.drop(columns=['mvrv_usd/bitcoin','realized_value_usd/bitcoin'],inplace=True)

In [53]:
#Change the index of metrics_df to the same of btc_df
metrics_df.index=metrics_df.index.tz_localize(None)

In [54]:
#Merge all dataframes
btc_total=(btc_df.merge(sp500_df,how='left',left_index=True,right_index=True)
.merge(nasdaq_df,how='left',left_index=True,right_index=True)
.merge(dow_df,how='left',left_index=True,right_index=True)
.merge(gold_df,how='left',left_index=True,right_index=True)
.merge(metrics_df,how='left',left_index=True,right_index=True))

In [58]:
btc_total.head()

,C_btc,V_btc,C_sp500,V_sp500,C_nasdaq_df,V_nasdaq_df,C_dow_df,V_dow_df,C_gold_df,V_gold_df,...,percent_of_total_supply_on_exchanges/bitcoin,social_dominance_total/bitcoin,sentiment_positive_total/bitcoin,sentiment_negative_total/bitcoin,sentiment_volume_consumed_total/bitcoin,circulation/bitcoin,daily_active_addresses/bitcoin,mean_age/bitcoin,mvrv_long_short_diff_usd/bitcoin,velocity/bitcoin
Date,,,,,,,,,,,,,,,,,,,,,
2013-04-28,134.210007,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1.109124e+07,118017.0,NaN,NaN,NaN
2013-04-29,144.539993,0.0,1593.609985,2.891200e+09,25.021957,1303500.0,14818.750000,97060000.0,1467.400024,269.0,...,NaN,NaN,NaN,NaN,NaN,1.109559e+07,86925.0,NaN,NaN,NaN
2013-04-30,139.000000,0.0,1597.569946,3.745070e+09,25.348705,1748200.0,14839.799805,148250000.0,1472.199951,534.0,...,NaN,NaN,NaN,NaN,NaN,1.109932e+07,76871.0,NaN,NaN,NaN
2013-05-01,116.989998,0.0,1582.699951,3.530320e+09,25.150936,919400.0,14700.950195,112620000.0,1446.300049,131.0,...,NaN,NaN,NaN,NaN,NaN,1.110309e+07,83565.0,NaN,NaN,NaN
2013-05-02,105.209999,0.0,1597.589966,3.366950e+09,25.770039,1436500.0,14831.580078,91180000.0,1467.699951,244.0,...,NaN,NaN,NaN,NaN,NaN,1.110644e+07,81920.0,NaN,NaN,NaN


In [59]:
btc_total.tail()

,C_btc,V_btc,C_sp500,V_sp500,C_nasdaq_df,V_nasdaq_df,C_dow_df,V_dow_df,C_gold_df,V_gold_df,...,percent_of_total_supply_on_exchanges/bitcoin,social_dominance_total/bitcoin,sentiment_positive_total/bitcoin,sentiment_negative_total/bitcoin,sentiment_volume_consumed_total/bitcoin,circulation/bitcoin,daily_active_addresses/bitcoin,mean_age/bitcoin,mvrv_long_short_diff_usd/bitcoin,velocity/bitcoin
Date,,,,,,,,,,,,,,,,,,,,,
2022-04-22,39740.319854,2.801172e+10,4271.779785,3.930660e+09,166.850006,830400.0,33811.398438,373280000.0,1931.000000,178.0,...,NaN,NaN,NaN,NaN,NaN,1.914098e+07,946969.0,NaN,NaN,NaN
2022-04-23,39486.728711,1.613802e+10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1.914200e+07,857710.0,NaN,NaN,NaN
2022-04-24,39469.293556,1.796440e+10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1.914307e+07,808655.0,NaN,NaN,NaN
2022-04-25,40458.308965,3.544573e+10,4296.120117,4.061070e+09,166.639999,1177800.0,34049.460938,416900000.0,1893.199951,748.0,...,NaN,NaN,NaN,NaN,NaN,1.914410e+07,1002972.0,NaN,NaN,NaN
2022-04-26,38117.462508,3.456909e+10,4175.200195,3.760420e+09,161.669998,899900.0,33240.179688,400020000.0,1901.400024,748.0,...,NaN,NaN,NaN,NaN,NaN,1.914498e+07,962904.0,NaN,NaN,NaN


In [60]:
btc_total.isnull().sum()

C_btc                                              0
V_btc                                              0
C_sp500                                         1020
V_sp500                                         1020
C_nasdaq_df                                     1020
V_nasdaq_df                                     1020
C_dow_df                                        1020
V_dow_df                                        1020
C_gold_df                                       1022
V_gold_df                                       1022
active_addresses_24h/bitcoin                       0
percent_of_total_supply_on_exchanges/bitcoin    2585
social_dominance_total/bitcoin                  2585
sentiment_positive_total/bitcoin                2585
sentiment_negative_total/bitcoin                2585
sentiment_volume_consumed_total/bitcoin         2585
circulation/bitcoin                                0
daily_active_addresses/bitcoin                     0
mean_age/bitcoin                              

In [92]:
#Create lags
btc_total['Lag_1']=btc_total['C_btc'].shift(1)
btc_total['Lag_2']=btc_total['C_btc'].shift(2)
btc_total['Lag_3']=btc_total['C_btc'].shift(3)
btc_total['Lag_-1']=btc_total['C_btc'].shift(-1)

#Create time columns
btc_total['Day_week']=np.array(btc_total.reset_index()['Date'].dt.dayofweek)
btc_total['Day_year']=np.array(btc_total.reset_index()['Date'].dt.dayofyear)
btc_total['Day_month']=np.array(btc_total.reset_index()['Date'].dt.day)
btc_total['year']=np.array(btc_total.reset_index()['Date'].dt.year)
btc_total['Month']=np.array(btc_total.reset_index()['Date'].dt.month)

#Create rolling mean
btc_total['rolling_mean_200']=btc_total['C_btc'].rolling(200).mean()


#Create mayer multiple
btc_total['price/ma200']=btc_total['C_btc']/btc_total['rolling_mean_200']

In [93]:
btc_total.isnull().sum()

C_btc                                              0
V_btc                                              0
C_sp500                                         1020
V_sp500                                         1020
C_nasdaq_df                                     1020
V_nasdaq_df                                     1020
C_dow_df                                        1020
V_dow_df                                        1020
C_gold_df                                       1022
V_gold_df                                       1022
active_addresses_24h/bitcoin                       0
percent_of_total_supply_on_exchanges/bitcoin    2585
social_dominance_total/bitcoin                  2585
sentiment_positive_total/bitcoin                2585
sentiment_negative_total/bitcoin                2585
sentiment_volume_consumed_total/bitcoin         2585
circulation/bitcoin                                0
daily_active_addresses/bitcoin                     0
mean_age/bitcoin                              

In [94]:
btc_total.head()

,C_btc,V_btc,C_sp500,V_sp500,C_nasdaq_df,V_nasdaq_df,C_dow_df,V_dow_df,C_gold_df,V_gold_df,...,Lag_2,Lag_3,Day_week,Day_year,Day_month,year,Month,rolling_mean_200,price/ma200,Lag_-1
Date,,,,,,,,,,,,,,,,,,,,,
2013-04-28,134.210007,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,6,118,28,2013,4,NaN,NaN,144.539993
2013-04-29,144.539993,0.0,1593.609985,2.891200e+09,25.021957,1303500.0,14818.750000,97060000.0,1467.400024,269.0,...,NaN,NaN,0,119,29,2013,4,NaN,NaN,139.000000
2013-04-30,139.000000,0.0,1597.569946,3.745070e+09,25.348705,1748200.0,14839.799805,148250000.0,1472.199951,534.0,...,134.210007,NaN,1,120,30,2013,4,NaN,NaN,116.989998
2013-05-01,116.989998,0.0,1582.699951,3.530320e+09,25.150936,919400.0,14700.950195,112620000.0,1446.300049,131.0,...,144.539993,134.210007,2,121,1,2013,5,NaN,NaN,105.209999
2013-05-02,105.209999,0.0,1597.589966,3.366950e+09,25.770039,1436500.0,14831.580078,91180000.0,1467.699951,244.0,...,139.000000,144.539993,3,122,2,2013,5,NaN,NaN,97.750000


In [95]:
btc_total.tail()

,C_btc,V_btc,C_sp500,V_sp500,C_nasdaq_df,V_nasdaq_df,C_dow_df,V_dow_df,C_gold_df,V_gold_df,...,Lag_2,Lag_3,Day_week,Day_year,Day_month,year,Month,rolling_mean_200,price/ma200,Lag_-1
Date,,,,,,,,,,,,,,,,,,,,,
2022-04-22,39740.319854,2.801172e+10,4271.779785,3.930660e+09,166.850006,830400.0,33811.398438,373280000.0,1931.000000,178.0,...,41374.378201,41502.750419,4,112,22,2022,4,47974.999298,0.828355,39486.728711
2022-04-23,39486.728711,1.613802e+10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,40527.364084,41374.378201,5,113,23,2022,4,47914.858871,0.824102,39469.293556
2022-04-24,39469.293556,1.796440e+10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,39740.319854,40527.364084,6,114,24,2022,4,47835.398091,0.825106,40458.308965
2022-04-25,40458.308965,3.544573e+10,4296.120117,4.061070e+09,166.639999,1177800.0,34049.460938,416900000.0,1893.199951,748.0,...,39486.728711,39740.319854,0,115,25,2022,4,47768.659710,0.846963,38117.462508
2022-04-26,38117.462508,3.456909e+10,4175.200195,3.760420e+09,161.669998,899900.0,33240.179688,400020000.0,1901.400024,748.0,...,39469.293556,39486.728711,1,116,26,2022,4,47689.407789,0.799286,NaN
